Tristan MANIER - Léa GRASSIEN - Mathieu RAMEL - INFO 5 - 9/01/2024


# 1. Présentation des données

Pourquoi avons nous choisis ces données ?
TODO

source, conditions de recueil, nombre de fichiers, nombres de variables et d'individus, expliquer l'activité derrière les données.
TODO

Donnez la signification desindividus et des variables
TODO

## Questions

### Analyse
Nous allons chercher à répondre à ces questions :
- Quelles sont les sources de production d'énergie les plus corrélées ?
- Quelles est la proportion d'énergie renouvelable produite ?
- Quelles sont les tendances de méthode de production d'énergie au cours de l'année ?
- Quelles sont les périodes de l'année où la production d'énergie est la plus importante/faible ?
- Quelles sont les moments de la journée où la production d'énergie est la plus importante/faible ?
- La météo influe-t-elle sur la consomation d'énergie ?
- Quelles sont les sources de production d'énergie les plus corrélées avec la météo ?

### Prédictions
- Quelle est la tendance de production d'énergie pour les jours/mois/années à venir ?
- 

# 2. Préparation des données

Ce notebook présente les commandes utiles pour les principales tâches de préparation des données. Les données `notes.data` qui servent d'illustration sont des notes obtenues par des étudiants dans des matières.

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 100)     # Set the maximum number of rows displayed to 100 rows

import math

## 2.1 Premier dataset sur la production d'énergie

In [ ]:
df = pd.read_csv("eCO2mix_RTE_En-cours-TR.csv", sep="\t", encoding='latin-1', index_col=False)

df.head()

In [ ]:
print(f"{df.shape[0]} rows and {df.shape[1]} columns")

# Combine dtypre, count and nnunique
pd.concat([df.dtypes, df.count(), df.nunique()], keys=["Types", "Count", "NUnique"], axis=1)

### Statistiques descriptives des valeurs non-définies

In [ ]:
temp = ({
    'column':[],
    'nb_lines' :[],
    'nb_ND':[]
})

nd = pd.DataFrame(temp)

for column in df.columns : 
    new_row = pd.DataFrame({'column': column, 'nb_lines': df[column].count(), 'nb_ND': (df[column] == 'ND').sum()}, index=[0])
    nd = pd.concat([new_row, nd.loc[:]]).reset_index(drop=True)
nd = nd.sort_values(by='nb_ND', ascending=False)

nd["%ND"] = nd["nb_ND"] / nd["nb_lines"] * 100

nd

## 2.2 Deuxième dataset sur les données météo

In [ ]:
df_weather = pd.read_csv("donnees-synop-essentielles-omm.csv", sep=";", index_col=False, encoding='utf-8')

# df_weather.head()
print(f"{df_weather.shape[0]} rows and {df_weather.shape[1]} columns")

df_weather.head()

Selection des données d'une seule région. (Pourais être remplacer par une moyenne des régions) 

In [ ]:
code_region = int('07149')  # Ile-de-France
df_weather = df_weather[df_weather["numer_sta"] == code_region]

# df_weather.head()
print(f"{df_weather.shape[0]} rows and {df_weather.shape[1]} columns")

Formattage des données météo.

Parmis toutes les données météo fournies, nous avons choisis de ne garder que les données les plus pertinantes pour nous. De plus, nous les avons renomer pour facilité leur utilisation et leur lisibilité.

In [ ]:
#multiplier les colonnes par 3 pour avoir 1 lignes par heures
weather_rename_map = {
    'date': 'date',
    'pmer': 'pression_mer',
    'ff': 'vitesse_vent',
    't': 'température',
    'u': 'humidité',
    'pres': 'pression',
    'niv_bar': 'niveau_barometrique',
    'tn12': 't_min_12h',
    'tn24': 't_min_24h',
    'tx12': 't_max_12h',
    'tx24': 't_max_24h',
    'tminsol': 't_min_sol_12h',
    'rr1': 'précipitation_1h',
    'rr3': 'précipitation_3h',
    'rr6': 'précipitation_6h',
    'rr12': 'précipitation_12h',
    'rr24': 'précipitation_24h',
    'ssfrai': 'hauteur_neige'
}

df_weather = df_weather.rename(columns=weather_rename_map)

# remove columns not in weather_rename_map
for col_name in df_weather.columns.values.tolist():
    if col_name not in weather_rename_map.values():
        df_weather = df_weather.drop(col_name, axis=1)

df_weather = df_weather.reset_index(drop=True)

# convert object to float
for col_name in df_weather.columns.values.tolist():
    df_weather[col_name] = pd.to_numeric(df_weather[col_name], errors='coerce')

#df_weather.head()

#df_weather.dtypes

Suppresion des lignes avec des valeurs manquantes pour les dates et heures.

In [ ]:
df = df.dropna(subset=['Date', "Heures"])

## 2.3 Jointure des deux datasets

Formatage des dates et heures des deux datasets.

In [ ]:
# Concat date and hours
df['date'] = df['Date'].astype(str) + df['Heures'].astype(str)

# Convert to datetime
df['date'] = df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d%H:%M'))

df_weather["date"] = df_weather['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d%H%M%S'))

Fusion des deux datasets

In [ ]:
df_full = pd.merge(df, df_weather, how='inner', on = 'date' )

# full_df.head()
print(f"df : {df.shape[0]} rows and {df.shape[1]} columns")
print(f"df_weather : {df_weather.shape[0]} rows and {df_weather.shape[1]} columns")
print(f"df_full : {df_full.shape[0]} rows and {df_full.shape[1]} columns")

print(f"Data between {df_full['date'].min()} and {df_full['date'].max()}")

# 3. Analyse des données

Selection d'une periode de temps

In [ ]:
from matplotlib import pyplot as plt

#select rows between date1 and date 2
date1 = pd.to_datetime('2022-06-01')
date2 = pd.to_datetime('2023-12-01')

df_selection = df_full[(df_full['date'] >= date1) & (df_full['date'] < date2)]

Visualisation des données de consomations

In [ ]:
figure = plt.figure(figsize=(20,10))
plt.plot(df_selection["date"], df_selection["Consommation"], label=column)
plt.legend( loc='upper right', numpoints = 1 )
plt.xlabel('Date')
plt.ylabel('Production (MW)')
plt.show()

Visualisation des données de production

In [ ]:
to_plot = ["Charbon", "Eolien", "Solaire", "Nucléaire", "Hydraulique", "Bioénergies", "Pompage"]

figure = plt.figure(figsize=(20,10))
for column in to_plot :
    plt.plot(df_selection["date"], df_selection[column], label=column)
    plt.legend( loc='upper right', numpoints = 1 )

plt.xlabel('Date')
plt.ylabel('Production (MW)')
plt.show()


### Quelles sont les sources de production d'énergie les plus corrélées ?

Selections des colonnes liées à la production et description de celles-ci

In [ ]:
df_energies = df_full[["Charbon", "Gaz", "Fioul", "Nucléaire", "Eolien", "Solaire", "Hydraulique", "Bioénergies", "Consommation"]]

df_energies.describe()

Matrice de corrélation

In [ ]:
df_energies.corr()

#### Conclusion
TODO: Mathieu

### Quelles est la proportion d'énergie renouvelable produite ?

Calcul de la production globale d'énergie sur la période du dataset

In [ ]:
name_col =["Charbon", "Gaz", "Fioul", "Nucléaire", "Eolien", "Solaire", "Hydraulique", "Bioénergies"]

df_energies = df_full[name_col]

df_energies = df_energies.dropna()

total_energies = sum(df_energies.sum(axis=1))
print(f"Production total : {total_energies} MW")

Calcul des proportions de chaque source d'énergie

In [ ]:
sum_dic = {}
pourcent_dic = {}
for col in name_col:
    sum_dic[col] = sum(df_energies[col])
    pourcent_dic[col] = sum_dic[col]/total_energies*100
    
pourcent_df = pd.DataFrame.from_dict(pourcent_dic, orient='index', columns=['%'])
print(pourcent_df)

Aggrégation des sources d'énergie en 3 catégories : renouvelable, fossile et nucléaire

In [ ]:
renouvelable = ["Eolien","Solaire","Hydraulique","Bioénergies"]
pourcent_renouvelable = sum([pourcent_dic[nrj12] for nrj12 in renouvelable])
print(f"Pourcentage d'énergie renouvelable : {pourcent_renouvelable:.2f} %")

fossil = ["Charbon","Gaz","Fioul"]
pourcent_fossil = sum([pourcent_dic[nrj12] for nrj12 in fossil])
print(f"Pourcentage d'énergie fossile : {pourcent_fossil:.2f} %")

nuclear = ["Nucléaire"]
pourcent_nuclear = sum([pourcent_dic[nrj12] for nrj12 in nuclear])
print(f"Pourcentage d'énergie nucléaire : {pourcent_nuclear:.2f} %")

Les énergies renouvelables représente plus d'un quart de la production total d'énergie.

### Quelles sont les tendances de méthode de production d'énergie au cours de l'année ?
TODO: mathieu
separer données en plusieurs periode et les plot

### La météo influe-t-elle sur la consomation d'énergie ?


In [ ]:
col_names = ["Consommation", "température", "vitesse_vent", "humidité", "pression", "précipitation_24h", "hauteur_neige"]
# df_full.columns.values.tolist()

df_selection = df_full[col_names]
#removes lines with nan or None
df_selection = df_selection.dropna()

df_selection.describe()

Matrice de corrélation

In [ ]:
corr_matrix = df_selection.corr()

# for only the "Consommation" column sort by descending order of absolute values
pd.DataFrame(corr_matrix["Consommation"].sort_values(ascending=False, key=lambda x: abs(x)))

### Conclusion

On remarque que la température est inversement corrélée avec la consomation d'énergie. Cela est logique car plus il fait froid, plus les gens chauffent leur logement.
Cependant c'est la seule corrélation notable.

### Quelles sont les sources de production d'énergie les plus corrélées avec la météo ?

In [169]:
col_names_prod =["Charbon", "Gaz", "Fioul", "Nucléaire", "Eolien", "Solaire", "Hydraulique", "Bioénergies"]
col_names_weather = ["température", "vitesse_vent", "humidité", "pression", "précipitation_24h", "hauteur_neige"]

df_selection = df_full[col_names_prod + col_names_weather]
df_selection = df_selection.dropna()

df_selection.describe()

,Charbon,Gaz,Fioul,Nucléaire,Eolien,Solaire,Hydraulique,Bioénergies,température,vitesse_vent,humidité,pression,précipitation_24h,hauteur_neige
count,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000
mean,182.846318,3846.537676,196.538101,33523.229034,4933.463176,3125.917412,5682.726692,730.951043,287.212495,3.914049,71.564921,100562.490421,1.822095,0.000004
std,403.679060,2413.278162,147.575528,5941.525968,3652.194985,3581.603563,2496.196559,97.395568,7.734232,1.953747,18.777471,926.887533,4.457989,0.000206
min,0.000000,243.000000,107.000000,19774.000000,557.000000,0.000000,1578.000000,0.000000,267.050000,0.000000,14.000000,96290.000000,-0.100000,0.000000
25%,14.000000,2027.000000,137.000000,28702.000000,2093.000000,0.000000,3790.000000,674.000000,281.850000,2.500000,58.000000,100080.000000,0.000000,0.000000
50%,27.000000,3501.000000,164.000000,33816.000000,3624.000000,1518.000000,5300.000000,705.000000,286.550000,3.700000,75.000000,100630.000000,0.000000,0.000000
75%,47.000000,5286.000000,197.000000,38162.000000,6824.000000,5742.000000,7001.000000,783.000000,292.750000,5.200000,87.000000,101150.000000,1.600000,0.000000
max,1835.000000,9728.000000,1442.000000,45799.000000,16331.000000,12083.000000,16145.000000,1092.000000,309.050000,12.200000,100.000000,103070.000000,44.000000,0.010000


In [170]:
#create a dataset with with the correlation between the production and the weather for each energy
corr_dataset = pd.DataFrame()
for col_name in col_names_prod:
    current_df = df_selection[[col_name] + col_names_weather]
    corr_matrix = current_df.corr()

    corr_list = corr_matrix[col_name]
    
    corr_dataset.insert(0, col_name, corr_matrix[col_name])

corr_dataset.dropna()

,Bioénergies,Hydraulique,Solaire,Eolien,Nucléaire,Fioul,Gaz,Charbon
température,-0.107017,-0.426612,0.631816,-0.273598,-0.575566,-0.335713,-0.544218,-0.510578
vitesse_vent,-0.001051,0.064231,0.164533,0.639315,0.012601,-0.029842,-0.194520,-0.081827
humidité,0.023190,0.245030,-0.726843,0.273818,0.267622,0.143183,0.265001,0.201082
pression,0.083321,-0.036470,0.118623,-0.464342,0.094257,0.019468,0.245654,0.131551
précipitation_24h,0.053438,0.003623,-0.088554,0.213873,-0.078030,-0.039498,-0.093169,-0.100703
hauteur_neige,-0.012070,0.072871,-0.016321,-0.016969,0.027629,0.053495,0.050178,0.083407


### Conclusion

On remarque que la température influence toutes les produciont d'énérgie, quand il fait froid la production augmentent et inversement. Sauf pour le solaire qui est invers car on produit plus d'énergie solaire en été lorsque le soeil est haut.

L'autre corrélation notable est celle entre le solaire et l'humidité. Cela est logique car il y a plus de nuage lorsque l'humidité est élevée et donc moins de soleil. De même pour les temps pluvieux. Cependant la corrélation entre le solaire et les précipitations des dernières 24h est inexistante, ce qui est très étonnant.

L'humidité à également un impact sur la production d'énégie hydrolique. Cela est logique car il y a plus d'eau dans les barages. Cependant elle a également in impact sur l'éolien, le nucléaire, le gaz et le charbon. Cela est plus difficile à expliquer.

La dernière corrélation est entre la vitesse du vent, la pression et la production d'énérgie éolienne qui est tout à fait logique, les pression et dépression entrainne du vent qui favorisent la production d'énergie éolienne.

# 4. Prédiction

In [ ]:
df.query('Charbon > 16').head()